In [1]:
import pandas as pd
import numpy as np
import os
import json
import csv

TL_sentence_path = r'C:\Users\edcrf\sentence_dataTL.csv'
VL_sentence_path = r'C:\Users\edcrf\sentence_dataVL.csv'

# data파일 불러오기
TL_sentence_data = pd.read_csv(TL_sentence_path, encoding='utf-8')
VL_sentence_data = pd.read_csv(VL_sentence_path, encoding='utf-8')

# 중복 제거, Pronuncication 열은 필요 없다고 생각
TL_sentence_data.drop('Pronunciation', axis=1, inplace=True)
TL_sentence_data = TL_sentence_data.drop_duplicates().reset_index(drop=True)
VL_sentence_data.drop('Pronunciation', axis=1, inplace=True)
VL_sentence_data = VL_sentence_data.drop_duplicates().reset_index(drop=True)

In [2]:
from tqdm import tqdm

# 형태소 분석기로 Okt를 사용
import konlpy
from konlpy.tag import Kkma, Komoran, Okt, Hannanum
okt = Okt()

standard_sentences = TL_sentence_data['Standard']
dialect_sentences = TL_sentence_data['Dialect']

In [3]:
# 방언 문장과 표준어 문장 각각 형태소 분석 진행
standard_okt = []
dialect_okt = []

stop_words = ['이', '에', '는', '가', '도', '을', '뭐', '은','하고','게','에는','그', '를', '것', '으로','로']

for i in tqdm(range(0, len(TL_sentence_data))):
    standard_tokens = [token for token in okt.morphs(standard_sentences[i]) if token not in stop_words]
    dialect_tokens = [token for token in okt.morphs(dialect_sentences[i]) if token not in stop_words]
    standard_okt.append(standard_tokens)
    dialect_okt.append(dialect_tokens)

100%|██████████| 30/30 [00:01<00:00, 21.75it/s]


In [6]:
standard_okt[0]

['여기는', '옥수수', '잘', '된다', '하던데', '옥수수', '말고는', '무슨', '농사', '많이', '짓습니까']

In [7]:
dialect_okt[0]

['여기는',
 '옥수',
 '갱',
 '잘',
 '된다',
 '카던디',
 '옥수',
 '갱',
 '말고는',
 '무신',
 '농사',
 '많이',
 '짓습',
 '니',
 '껴']

In [79]:
# 토큰화된 방언 문장과 표준어 문장을 각각 csv파일로 저장
with open('st_okt.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(standard_okt)

with open('di_okt.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(dialect_okt)

In [74]:
# # csv저장을 위해 리스트를 DataFrame으로 변환
# standard_okt_df = pd.DataFrame(standard_okt)
# dialect_okt_df = pd.DataFrame(dialect_okt)

# standard_okt_df.to_csv(r'C:\Users\edcrf\standard_okt.csv', index=False, sep =',')
# dialect_okt_df.to_csv(r'C:\Users\edcrf\dialect_okt.csv', index=False, sep =',')

In [ ]:
# # csv파일로 저장한 토큰화된 결과 불러오기
# standard_okt_data = pd.read_csv(r'C:\Users\edcrf\standard_okt.csv')
# dialect_okt_data = pd.read_csv(r'C:\Users\edcrf\dialect_okt.csv')

In [5]:
# 저장된 토큰화 데이터 불러오기
standard_okt_data = []
dialect_okt_data = []

with open('st_okt.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        standard_okt_data.append(row)

with open('di_okt.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        dialect_okt_data.append(row)

In [6]:
# 방언 문장의 토큰과 표준어 문장의 토큰 저장
dialect_tokens = []
standard_tokens = []

for i in range(0, len(TL_sentence_data)):
    for standard_word in standard_okt_data[i]:
        standard_tokens.append(standard_word)
    
    for dialect_word in dialect_okt_data[i]:
        dialect_tokens.append(dialect_word)

In [7]:
# 각 리스트에서 중복을 제거
unique_standard_tokens = []
unique_dialect_tokens = []
seen1 = set()
seen2 = set()

for word in standard_tokens:
    if word not in seen1:
        unique_standard_tokens.append(word)
        seen1.add(word)

for word in dialect_tokens:
    if word not in seen2:
        unique_dialect_tokens.append(word)
        seen2.add(word)


In [11]:
print(f"중복 제거 전 토큰 수 >> standard: {len(standard_tokens)}, dialect: {len(dialect_tokens)}")
print(f"중복 제거 후 토큰 수 >> standard: {len(unique_standard_tokens)}, dialect: {len(unique_dialect_tokens)}")

중복 제거 전 토큰 수 >> standard: 6536600, dialect: 6794654
중복 제거 후 토큰 수 >> standard: 91751, dialect: 111884


In [8]:
tokens = standard_tokens + dialect_tokens

print(len(tokens))

13331254


In [9]:
all_tokens = standard_tokens + dialect_tokens

unique_all_tokens = []
seen3 = set()

for word in all_tokens:
    if word not in seen3:
        unique_all_tokens.append(word)
        seen3.add(word)

In [21]:
print(f"중복 제거 전 방언과 표준어의 토큰 합친 수: {len(all_tokens)}")
print(f"중복 제거 후 방언과 표준어의 토큰 합친 수: {len(unique_all_tokens)}")

중복 제거 전 방언과 표준어의 토큰 합친 수: 13331254
중복 제거 후 방언과 표준어의 토큰 합친 수: 120152


### 중복 제거 x -> standard: 6536600, dialect: 6794654
#### => 전체 토큰 수: 13331254
### 중복 제거 o -> standard: 91751, dialect: 111884
#### 전체 토큰 수(중복 제거 x): 13331254, (중복 제거 o): 120152

In [16]:
from collections import Counter

# 중복을 하나도 제거하지 않았을 때의 단어 빈도수를 계산
word_counts = Counter(tokens)

# 빈도수 높은 순서대로 정렬
sorted_word = sorted(word_counts, key=word_counts.get, reverse=True)

print(len(sorted_word))

120152


In [22]:
sorted_word[120130:120152]

['고란',
 '노으면서',
 '싫니',
 '질퍽질퍽해',
 '씰어져',
 '뛰야',
 '땡긴다아이가',
 '묵어지겠지',
 '서가있을',
 '걸렸는거라',
 '보인데이',
 '올러오면',
 '보이겠나',
 '매져가',
 '축축해가',
 '찍을까',
 '것뱈',
 '골르라',
 '골라라면',
 '버려뻐려',
 '흘르니까',
 '해믹일꼬']

In [26]:
### 단어 빈도수가 높은 것들을 출력해보니 방언과 표준어의 관계와 전혀 관련없는 조사들이 많이 포함됨
### 따라서 불용어 처리를 통해 제거가 필요함
# stop_words = ['이', '에', '는', '가', '도', '을', '뭐', '은','하고','게','에는','그', '를', '것', '으로','로']